<a href="https://colab.research.google.com/github/wnyngj/class2022spring/blob/main/requests_gradio0328.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr
import pandas as pd

import requests


In [2]:
#url로 request하는 것 - rest api
URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/' #서비스URL
      'getCtprvnMesureSidoLIst?'  #세부서비스 + ?
      'sidoName=서울' #시도 명
      '&searchCondition=hour' #&데이터 기간
      '&pageNo=1' #&페이지넘버
      '&numOfRows=100'  #&row는 100개
      '&returnType=json'  #&json형태로 받고싶다
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
#&서비스 키: 각자 가입해서 받는 고유 번호
response = requests.get(URL)
#response 형태는 json

In [3]:
print(response)

<Response [200]>


#### http://jsonviewer.stack.hu/ (copy json here to view json)

ㄴjason의 형태로 왓는데 이 구조가 무엇일지 궁금하면 response.json()값을 저기에 붙여넣으면 분석해줌

In [ ]:
r = response.json()   
r

In [4]:
items = r['response']['body']['items']

#정보들 list로 뽑기
cityName = [item['cityName'] for item in items] #
pm10Value = [item['pm10Value'] for item in items]
pm25Value = [item['pm25Value'] for item in items]
so2Value = [item['so2Value'] for item in items]
coValue = [item['coValue'] for item in items]
o3Value = [item['o3Value'] for item in items]
no2Value = [item['no2Value'] for item in items]

#pandas에서 column별로 series 만들기
pd_cityName = pd.Series(cityName)  
pd_pm10Value = pd.Series(pm10Value)   
pd_pm25Value = pd.Series(pm25Value)
so2Value = pd.Series(so2Value)
coValue = pd.Series(coValue)
o3Value = pd.Series(o3Value)
no2Value = pd.Series(no2Value)

#엑셀파일 형태로 df만들기, response 받아오기
df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})

gradio로 웹데모를 만들려면 function의 형태로 있어야함

In [6]:
def test(sidoName):#입력 sidoName
  URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'
      'getCtprvnMesureSidoLIst?'
      'sidoName='+sidoName+
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
  
  response = requests.get(URL)
  r = response.json()
  items = r['response']['body']['items']

  cityName = [item['cityName'] for item in items]
  pm10Value = [item['pm10Value'] for item in items]
  pm25Value = [item['pm25Value'] for item in items]
  so2Value = [item['so2Value'] for item in items]
  coValue = [item['coValue'] for item in items]
  o3Value = [item['o3Value'] for item in items]
  no2Value = [item['no2Value'] for item in items]

  pd_cityName = pd.Series(cityName)
  pd_pm10Value = pd.Series(pm10Value)   
  pd_pm25Value = pd.Series(pm25Value)
  so2Value = pd.Series(so2Value)
  coValue = pd.Series(coValue)
  o3Value = pd.Series(o3Value)
  no2Value = pd.Series(no2Value)

  df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})
  return df

In [ ]:
iface = gr.Interface(test, gr.inputs.Dropdown(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종']), outputs=['dataframe'])
#입력에 dropdown형태로 넣고 출력에 dataframe형태로
iface.launch(debug=True)